# **Model Evaluation**

# NB Sentiment Analysis Predictor

The "NB Sentiment Analysis Predictor" is a sophisticated tool developed in Python, employing Natural Language Processing (NLP) and Machine Learning (ML) techniques. Its primary function is to classify the sentiment of input text, using a Naive Bayes (NB) classifier trained on the IMDB movie reviews dataset. The program can discern between positive and negative sentiments with high accuracy.

## Key Features:

1. **Advanced Text Processing:** The tool incorporates basic preprocessing techniques, including lowercasing, HTML tag removal, handling negations, expanding acronyms, and tokenization. This ensures optimal preparation of input text for analysis.
2. **TF-IDF Vectorization:** It utilises Term Frequency-Inverse Document Frequency (TF-IDF) vectorization, transforming text data into a numerical format for effective processing and analysis by the Naive Bayes model.
3. **User-Friendly Interface:** The program offers a simple and intuitive interface, accessible to both technical and non-technical users.
4. **Real-Time Analysis:** It processes input text promptly and predicts sentiment in real time.
5. **High Accuracy and Reliability:** The model's training on diverse datasets ensures reliable and accurate sentiment predictions.

## Applications:

- **Customer Feedback Analysis:** Suitable for analysing customer reviews and feedback.
- **Social Media Monitoring:** Helps in monitoring social media posts and comments for public opinion and trends.
- **Content Moderation:** Useful in identifying negative sentiments for online content moderation.

## Flexibility:

The "NB Sentiment Predictor" is adaptable, allowing for future enhancements with evolving ML and deep learning (DL) technologies:

- **Model Adaptability:** The current Naive Bayes classifier can be replaced with other ML algorithms or advanced DL models.
- **Text Representation Evolution:** Supports alternative text representation techniques, including Bag of Words, Word Embeddings, or transformer-based models like BERT.

## Deployment:

The program can be deployed as a web application, enabling users to access the tool via a web browser for instant sentiment analysis results. This deployment method ensures broad reach and ease of use.
os.for various scenarios.


# **Prerequisites**

**Install Required Packages**

In [1]:
# install the 'datasets' library
!pip install datasets -q

# install the 'spacy' library
!pip install spacy -q

# installing the joblib library
!pip install joblib -q


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#downloads the small English model (en_core_web_sm) for the Spacy library.
!python -m spacy download en_core_web_sm -q

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


**Load Dataset**

In [5]:
# Import necessary libraries
import pandas as pd
from datasets import load_dataset

# load the IMDB dataset
dataset = load_dataset('imdb')

# extract train and test datasets
train_dataset = dataset['train']
test_dataset = dataset['test']

# convert to pandas DataFrames
trainData = pd.DataFrame(train_dataset)
testData = pd.DataFrame(test_dataset)


# **Dataset Analysis**

In [6]:
# call the 'head()' method on the 'trainData' DataFrame to inspect the first five rows.

trainData.head()

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


In [7]:
# call the 'head()' method on the 'testData' DataFrame to inspect the first five rows.

testData.head()

,text,label
0,I love sci-fi and am willing to put up with a ...,0
1,"Worth the entertainment value of a rental, esp...",0
2,its a totally average film with a few semi-alr...,0
3,STAR RATING: ***** Saturday Night **** Friday ...,0
4,"First off let me say, If you haven't enjoyed a...",0


**Remove Duplicates**

In [8]:
# calculate the number of duplicated entries in the 'text' column of the 'trainData' DataFrame.

trainDataDuplicates = trainData['text'].duplicated().sum()
trainDataDuplicates

96

In [9]:
# remove duplicate rows from 'trainData' based on the 'text' column.
noTrainDataDuplicates = trainData.drop_duplicates(subset='text')
noTrainDataDuplicatesShape = noTrainDataDuplicates.shape

noTrainDataDuplicatesShape

(24904, 2)

In [10]:
# calculate the number of duplicated entries in the 'text' column of the 'testData' DataFrame.

testDataDuplicates = testData['text'].duplicated().sum()
testDataDuplicates

199

In [11]:
# remove duplicate rows from 'testData' based on the 'text' column.

noTestDataDuplicates = testData.drop_duplicates(subset='text')
noTestDataDuplicatesShape = noTestDataDuplicates.shape

noTestDataDuplicatesShape

(24801, 2)

In [12]:
# create copies of the noTrainDataDuplicates and noTestDataDuplicates DataFrames.


rawtrainData = noTrainDataDuplicates.copy()
rawtestData = noTestDataDuplicates.copy()

In [13]:
import pandas as pd
import spacy
from bs4 import BeautifulSoup
import re

# load SpaCy model
nlp = spacy.load('en_core_web_sm')


# **Basic Preprocessing**

In [15]:
# this function is designed to perform basic text preprocessing tasks.

def preprocess_basic(text):
    # convert text to lowercase
    text = text.lower()

    # remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # handle negations (e.g., isn't -> is not)
    contractions_dict = {
        "ain't": "am not / are not / is not / has not / have not",
        "aren't": "are not / am not",
        "can't": "cannot",
        "can't've": "cannot have",
        "'cause": "because",
        "could've": "could have",
        "couldn't": "could not",
        "couldn't've": "could not have",
        "didn't": "did not",
        "doesn't": "does not",
        "don't": "do not",
        "hadn't": "had not",
        "hadn't've": "had not have",
        "hasn't": "has not",
        "haven't": "have not",
        "he'd": "he had / he would",
        "he'd've": "he would have",
        "he'll": "he shall / he will",
        "he'll've": "he shall have / he will have",
        "he's": "he has / he is",
        "how'd": "how did",
        "how'd'y": "how do you",
        "how'll": "how will",
        "how's": "how has / how is / how does",
        "I'd": "I had / I would",
        "I'd've": "I would have",
        "I'll": "I shall / I will",
        "I'll've": "I shall have / I will have",
        "I'm": "I am",
        "I've": "I have",
        "isn't": "is not",
        "it'd": "it had / it would",
        "it'd've": "it would have",
        "it'll": "it shall / it will",
        "it'll've": "it shall have / it will have",
        "it's": "it has / it is",
        "let's": "let us",
        "ma'am": "madam",
        "mayn't": "may not",
        "might've": "might have",
        "mightn't": "might not",
        "mightn't've": "might not have",
        "must've": "must have",
        "mustn't": "must not",
        "mustn't've": "must not have",
        "needn't": "need not",
        "needn't've": "need not have",
        "o'clock": "of the clock",
        "oughtn't": "ought not",
        "oughtn't've": "ought not have",
        "shan't": "shall not",
        "sha'n't": "shall not",
        "shan't've": "shall not have",
        "she'd": "she had / she would",
        "she'd've": "she would have",
        "she'll": "she shall / she will",
        "she'll've": "she shall have / she will have",
        "she's": "she has / she is",
        "should've": "should have",
        "shouldn't": "should not",
        "shouldn't've": "should not have",
        "so've": "so have",
        "so's": "so as / so is",
        "that'd": "that would / that had",
        "that'd've": "that would have",
        "that's": "that has / that is",
        "there'd": "there had / there would",
        "there'd've": "there would have",
        "there's": "there has / there is",
        "they'd": "they had / they would",
        "they'd've": "they would have",
        "they'll": "they shall / they will",
        "they'll've": "they shall have / they will have",
        "they're": "they are",
        "they've": "they have",
        "to've": "to have",
        "wasn't": "was not",
        "we'd": "we had / we would",
        "we'd've": "we would have",
        "we'll": "we will",
        "we'll've": "we will have",
        "we're": "we are",
        "we've": "we have",
        "weren't": "were not",
        "what'll": "what shall / what will",
        "what'll've": "what shall have / what will have",
        "what're": "what are",
        "what's": "what has / what is",
        "what've": "what have",
        "when's": "when has / when is",
        "when've": "when have",
        "where'd": "where did",
        "where's": "where has / where is",
        "where've": "where have",
        "who'll": "who shall / who will",
        "who'll've": "who shall have / who will have",
        "who's": "who has / who is",
        "who've": "who have",
        "why's": "why has / why is",
        "why've": "why have",
        "will've": "will have",
        "won't": "will not",
        "won't've": "will not have",
        "would've": "would have",
        "wouldn't": "would not",
        "wouldn't've": "would not have",
        "y'all": "you all",
        "y'all'd": "you all would",
        "y'all'd've": "you all would have",
        "y'all're": "you all are",
        "y'all've": "you all have",
        "you'd": "you had / you would",
        "you'd've": "you would have",
        "you'll": "you shall / you will",
        "you'll've": "you shall have / you will have",
        "you're": "you are",
        "you've": "you have"
    }
    contractions_pattern = re.compile(r'\b(' + '|'.join(contractions_dict.keys()) + r')\b')
    text = contractions_pattern.sub(lambda x: contractions_dict[x.group()], text)

    # expand acronyms
    acronyms_dict = {
        'afaik': 'as far as i know',  
        'afk': 'away from keyboard',
        'asap': 'as soon as possible',
        'atk': 'at the keyboard',
        'atm': 'at the moment',
        'a3': 'anytime, anywhere, anyplace',
        'bak': 'back at keyboard',
        'bbl': 'be back later',
        'bbs': 'be back soon',
        'bfn': 'bye for now',
        'b4n': 'bye for now',
        'brb': 'be right back',
        'brt': 'be right there',
        'btw': 'by the way',
        'b4': 'before',
        'b4n': 'bye for now',
        'cu': 'see you',
        'cul8r': 'see you later',
        'cya': 'see you',
        'faq': 'frequently asked questions',
        'fc': 'fingers crossed',
        'fwiw': 'for what it\'s worth',
        'fyi': 'for your information',
        'gal': 'get a life',
        'gg': 'good game',
        'gn': 'good night',
        'gmta': 'great minds think alike',
        'gr8': 'great!',
        'g9': 'genius',
        'ic': 'i see',
        'icq': 'i seek you (also a chat program)',
        'ilu': 'ilu: i love you',
        'imho': 'in my honest/humble opinion',
        'imo': 'in my opinion',
        'iow': 'in other words',
        'irl': 'in real life',
        'kiss': 'keep it simple, stupid',
        'ldr': 'long distance relationship',
        'lmao': 'laugh my a.. off',
        'lol': 'laughing out loud',
        'ltns': 'long time no see',
        'l8r': 'later',
        'mte': 'my thoughts exactly',
        'm8': 'mate',
        'nrn': 'no reply necessary',
        'oic': 'oh i see',
        'pita': 'pain in the a..',
        'prt': 'party',
        'prw': 'parents are watching',
        'qpsa?': 'que pasa?',
        'rofl': 'rolling on the floor laughing',
        'roflol': 'rolling on the floor laughing out loud',
        'rotflmao': 'rolling on the floor laughing my a.. off',
        'sk8': 'skate',
        'stats': 'your sex and age',
        'asl': 'age, sex, location',
        'thx': 'thank you',
        'ttfn': 'ta-ta for now!',
        'ttyl': 'talk to you later',
        'u': 'you',
        'u2': 'you too',
        'u4e': 'yours for ever',
        'wb': 'welcome back',
        'wtf': 'what the f...',
        'wtg': 'way to go!',
        'wuf': 'where are you from?',
        'w8': 'wait...',
        '7k': 'sick:-d laugher',
        'tfw': 'that feeling when',
        'mfw': 'my face when',
        'mrw': 'my reaction when',
        'ifyp': 'i feel your pain',
        'lol': 'laughing out loud',
        'tntl': 'trying not to laugh',
        'jk': 'just kidding',
        'idc': 'i don’t care',
        'ily': 'i love you',
        'imu': 'i miss you',
        'adih': 'another day in hell',
        'idc': 'i don’t care',
        'zzz': 'sleeping, bored, tired',
        'wywh': 'wish you were here',
        'time': 'tears in my eyes',
        'bae': 'before anyone else',
        'fimh': 'forever in my heart',
        'bsaaw': 'big smile and a wink',
        'bwl': 'bursting with laughter',
        'lmao': 'laughing my a** off',
        'bff': 'best friends forever',
        'csl': 'can’t stop laughing'
    }
    acronyms_pattern = re.compile(r'\b(' + '|'.join(acronyms_dict.keys()) + r')\b')
    text = acronyms_pattern.sub(lambda x: acronyms_dict[x.group()], text)

    # tokenize and remove punctuation
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_punct]

 # further cleaning (like removing non-alphabetic characters)
    cleaned_tokens = [token for token in tokens if token.isalpha()]

    cleaned_text = ' '.join(cleaned_tokens)

    return cleaned_text

In [17]:
rawtrainData['processed_text'] = rawtrainData['text'].apply(preprocess_basic)

C:\Users\momo\AppData\Local\Temp\ipykernel_30896\1535735849.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [18]:
rawtestData['processed_text'] = rawtestData['text'].apply(preprocess_basic)

C:\Users\momo\AppData\Local\Temp\ipykernel_30896\1535735849.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


# **Feature Extraction**

**Term Fequency-Inverse Document Frequency**

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

# return the fitted vectoriser
def create_ml_tfidf_representation(train_data, test_data):
    vectoriser = TfidfVectorizer()
    trainTfidf = vectoriser.fit_transform(rawtrainData['processed_text'])
    testTfidf = vectoriser.transform(rawtestData['processed_text'])
    return trainTfidf, testTfidf, vectoriser

trainTfidf, testTfidf, vectoriser = create_ml_tfidf_representation(rawtrainData, rawtestData)

# save the vectoriser
joblib.dump(vectoriser, 'tfidf_basic_vectoriser.pkl')

['tfidf_basic_vectoriser.pkl']

0        0
1        0
2        0
3        0
4        0
        ..
24995    1
24996    1
24997    1
24998    1
24999    1
Name: label, Length: 24904, dtype: int64

# **Classification**

In [25]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, f1_score
import joblib

# train the model
nb_model = MultinomialNB()
nb_model.fit(trainTfidf, rawtrainData['label'])

# evaluate the model
test_predictions = nb_model.predict(testTfidf)
f1 = f1_score(rawtestData['label'], test_predictions, average='binary') 
print(f"Accuracy: {accuracy_score(rawtestData['label'], test_predictions)}")
print(f"F1 Score: {f1}")
print(classification_report(rawtestData['label'], test_predictions))  

# save the trained model
joblib.dump(nb_model, 'sentiment_naive_bayes_model.pkl')


Accuracy: 0.8301278174267167
F1 Score: 0.8207615400978514
              precision    recall  f1-score   support

           0       0.80      0.89      0.84     12361
           1       0.87      0.78      0.82     12440

    accuracy                           0.83     24801
   macro avg       0.83      0.83      0.83     24801
weighted avg       0.83      0.83      0.83     24801



['sentiment_naive_bayes_model.pkl']

In [ ]:
import joblib

# load the trained model and vectorizer
nb_model = joblib.load('sentiment_naive_bayes_model.pkl')
vectorizer = joblib.load('tfidf_basic_vectoriser.pkl')

# load SpaCy model
nlp = spacy.load('en_core_web_sm')

# preprocessing function
def preprocess_basic(text):
    # convert text to lowercase
    text = text.lower()

    # remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # handle negations (e.g., isn't -> is not)
    contractions_dict = {
        "ain't": "am not / are not / is not / has not / have not",
        "aren't": "are not / am not",
        "can't": "cannot",
        "can't've": "cannot have",
        "'cause": "because",
        "could've": "could have",
        "couldn't": "could not",
        "couldn't've": "could not have",
        "didn't": "did not",
        "doesn't": "does not",
        "don't": "do not",
        "hadn't": "had not",
        "hadn't've": "had not have",
        "hasn't": "has not",
        "haven't": "have not",
        "he'd": "he had / he would",
        "he'd've": "he would have",
        "he'll": "he shall / he will",
        "he'll've": "he shall have / he will have",
        "he's": "he has / he is",
        "how'd": "how did",
        "how'd'y": "how do you",
        "how'll": "how will",
        "how's": "how has / how is / how does",
        "I'd": "I had / I would",
        "I'd've": "I would have",
        "I'll": "I shall / I will",
        "I'll've": "I shall have / I will have",
        "I'm": "I am",
        "I've": "I have",
        "isn't": "is not",
        "it'd": "it had / it would",
        "it'd've": "it would have",
        "it'll": "it shall / it will",
        "it'll've": "it shall have / it will have",
        "it's": "it has / it is",
        "let's": "let us",
        "ma'am": "madam",
        "mayn't": "may not",
        "might've": "might have",
        "mightn't": "might not",
        "mightn't've": "might not have",
        "must've": "must have",
        "mustn't": "must not",
        "mustn't've": "must not have",
        "needn't": "need not",
        "needn't've": "need not have",
        "o'clock": "of the clock",
        "oughtn't": "ought not",
        "oughtn't've": "ought not have",
        "shan't": "shall not",
        "sha'n't": "shall not",
        "shan't've": "shall not have",
        "she'd": "she had / she would",
        "she'd've": "she would have",
        "she'll": "she shall / she will",
        "she'll've": "she shall have / she will have",
        "she's": "she has / she is",
        "should've": "should have",
        "shouldn't": "should not",
        "shouldn't've": "should not have",
        "so've": "so have",
        "so's": "so as / so is",
        "that'd": "that would / that had",
        "that'd've": "that would have",
        "that's": "that has / that is",
        "there'd": "there had / there would",
        "there'd've": "there would have",
        "there's": "there has / there is",
        "they'd": "they had / they would",
        "they'd've": "they would have",
        "they'll": "they shall / they will",
        "they'll've": "they shall have / they will have",
        "they're": "they are",
        "they've": "they have",
        "to've": "to have",
        "wasn't": "was not",
        "we'd": "we had / we would",
        "we'd've": "we would have",
        "we'll": "we will",
        "we'll've": "we will have",
        "we're": "we are",
        "we've": "we have",
        "weren't": "were not",
        "what'll": "what shall / what will",
        "what'll've": "what shall have / what will have",
        "what're": "what are",
        "what's": "what has / what is",
        "what've": "what have",
        "when's": "when has / when is",
        "when've": "when have",
        "where'd": "where did",
        "where's": "where has / where is",
        "where've": "where have",
        "who'll": "who shall / who will",
        "who'll've": "who shall have / who will have",
        "who's": "who has / who is",
        "who've": "who have",
        "why's": "why has / why is",
        "why've": "why have",
        "will've": "will have",
        "won't": "will not",
        "won't've": "will not have",
        "would've": "would have",
        "wouldn't": "would not",
        "wouldn't've": "would not have",
        "y'all": "you all",
        "y'all'd": "you all would",
        "y'all'd've": "you all would have",
        "y'all're": "you all are",
        "y'all've": "you all have",
        "you'd": "you had / you would",
        "you'd've": "you would have",
        "you'll": "you shall / you will",
        "you'll've": "you shall have / you will have",
        "you're": "you are",
        "you've": "you have"
    }
    contractions_pattern = re.compile(r'\b(' + '|'.join(contractions_dict.keys()) + r')\b')
    text = contractions_pattern.sub(lambda x: contractions_dict[x.group()], text)

    # xxpand acronyms
    acronyms_dict = {
        'afaik': 'as far as i know',  
        'afk': 'away from keyboard',
        'asap': 'as soon as possible',
        'atk': 'at the keyboard',
        'atm': 'at the moment',
        'a3': 'anytime, anywhere, anyplace',
        'bak': 'back at keyboard',
        'bbl': 'be back later',
        'bbs': 'be back soon',
        'bfn': 'bye for now',
        'b4n': 'bye for now',
        'brb': 'be right back',
        'brt': 'be right there',
        'btw': 'by the way',
        'b4': 'before',
        'b4n': 'bye for now',
        'cu': 'see you',
        'cul8r': 'see you later',
        'cya': 'see you',
        'faq': 'frequently asked questions',
        'fc': 'fingers crossed',
        'fwiw': 'for what it\'s worth',
        'fyi': 'for your information',
        'gal': 'get a life',
        'gg': 'good game',
        'gn': 'good night',
        'gmta': 'great minds think alike',
        'gr8': 'great!',
        'g9': 'genius',
        'ic': 'i see',
        'icq': 'i seek you (also a chat program)',
        'ilu': 'ilu: i love you',
        'imho': 'in my honest/humble opinion',
        'imo': 'in my opinion',
        'iow': 'in other words',
        'irl': 'in real life',
        'kiss': 'keep it simple, stupid',
        'ldr': 'long distance relationship',
        'lmao': 'laugh my a.. off',
        'lol': 'laughing out loud',
        'ltns': 'long time no see',
        'l8r': 'later',
        'mte': 'my thoughts exactly',
        'm8': 'mate',
        'nrn': 'no reply necessary',
        'oic': 'oh i see',
        'pita': 'pain in the a..',
        'prt': 'party',
        'prw': 'parents are watching',
        'qpsa?': 'que pasa?',
        'rofl': 'rolling on the floor laughing',
        'roflol': 'rolling on the floor laughing out loud',
        'rotflmao': 'rolling on the floor laughing my a.. off',
        'sk8': 'skate',
        'stats': 'your sex and age',
        'asl': 'age, sex, location',
        'thx': 'thank you',
        'ttfn': 'ta-ta for now!',
        'ttyl': 'talk to you later',
        'u': 'you',
        'u2': 'you too',
        'u4e': 'yours for ever',
        'wb': 'welcome back',
        'wtf': 'what the f...',
        'wtg': 'way to go!',
        'wuf': 'where are you from?',
        'w8': 'wait...',
        '7k': 'sick:-d laugher',
        'tfw': 'that feeling when',
        'mfw': 'my face when',
        'mrw': 'my reaction when',
        'ifyp': 'i feel your pain',
        'lol': 'laughing out loud',
        'tntl': 'trying not to laugh',
        'jk': 'just kidding',
        'idc': 'i don’t care',
        'ily': 'i love you',
        'imu': 'i miss you',
        'adih': 'another day in hell',
        'idc': 'i don’t care',
        'zzz': 'sleeping, bored, tired',
        'wywh': 'wish you were here',
        'time': 'tears in my eyes',
        'bae': 'before anyone else',
        'fimh': 'forever in my heart',
        'bsaaw': 'big smile and a wink',
        'bwl': 'bursting with laughter',
        'lmao': 'laughing my a** off',
        'bff': 'best friends forever',
        'csl': 'can’t stop laughing'
    }
    acronyms_pattern = re.compile(r'\b(' + '|'.join(acronyms_dict.keys()) + r')\b')
    text = acronyms_pattern.sub(lambda x: acronyms_dict[x.group()], text)

    # tokenize and remove punctuation
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_punct]

 # further cleaning (like removing non-alphabetic characters)
    cleaned_tokens = [token for token in tokens if token.isalpha()]

    cleaned_text = ' '.join(cleaned_tokens)

    return cleaned_text


In [35]:
def predict_sentiment(user_input):
    preprocessed_text = preprocess_basic(user_input)
    vectorized_text = vectorizer.transform([ preprocessed_text])
    prediction = nb_model.predict(vectorized_text)
    return "Positive" if prediction[0] == 1 else "Negative"

# accept user input
input_text = input("Enter a sentence to analyse sentiment: ")

# predict and display the result
result = predict_sentiment(input_text)
print(f"The sentiment of the input is: {result}")

Enter a sentence to analyse sentiment:  Seems to be a trend making limited series that don't need 8-10 episodes! First episode gets you hooked but you see the signs of the long drawn-out development being framed.  Being fooled in the past from other shows, I now look for reviews that recommend the episodes to skip, that don't provide critical information. The reason for the 6 is the unnecessary episodes.  If you watch episode 1, and skip to last two 7,8... the rating goes up to a 8 for me. No critical information is missing and I like the ending, they tied up most of the plot lines.  Overall a great show, a little overly dramatized for a UK show which tends to be ground compared to US TV.


The sentiment of the input is: Positive
